In [0]:
# [1]
# Age Classifier with stratified Kfold cross validation, UTK
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt
import time

from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def stratified_sampling(X, y, test_size, random_state):
    # apply Scikit stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in sss.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test
  
print("###############<< Age Classifier >>###############\n")
start_time = time.time()

print("=========<< [1] Retrieving Data >>=========")

# Initialize GoogleDriveFile instance with file id.
data_file_64 = drive.CreateFile({'id': '1BE5HdE_mYkd6XjJ7e_pY6m3nfGaRZ87_'})
data_file_64.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_128 = drive.CreateFile({'id': '1Tsf7XwB09dUZVXSmlO81Fw3lumUnRiNb'})
# data_file_128.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_200 = drive.CreateFile({'id': '1hTnC0V1-P0sT-nLhl5JK7ETMpXdL9zgd'})
# data_file_200.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# train_data_128_adience = drive.CreateFile({'id': '17uW_i5i4QIsbka_QPE424B2OyZgKE7jV'})
# train_data_128_adience.GetContentFile('Adience_train_data.mat') 

# test_data_128_adience = drive.CreateFile({'id': '1nFm9J_rmZpI6ToVBeLrp1GPgXWVnamZT'})
# test_data_128_adience.GetContentFile('Adience_test_data.mat') 

num_cats = 8 # number of age categories (0, 4), [4, 8), [8, 15), [15, 25), [25, 38), [38, 48), [48, 60), 60+
sz = 64     # image dimension
print('UTK dataset: ', sz, " ===> Age Classification")
data = scipy.io.loadmat('UTKdata')
X = data['X']
y = data['y_age'][0]

print('=========<< [2] Splitting data into train, validate & test >>=========')

input_shape = (sz, sz, 3)

X, X_test, y, y_test = stratified_sampling(X, y, test_size=0.15, random_state=37)
y_test = np_utils.to_categorical(y_test, num_cats)

print("Train size = ", y.shape[0]) 
print("Test size = ", y_test.shape[0])

print('=========<< [3] Model build & train >>=========')

# create model
vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)
model = Sequential()
for new_layer in vgg16_model.layers:
    model.add(new_layer)
# print(model.summary())

# # load weigts trained on a previus training session
# last_weight_file = drive.CreateFile({'id': '1-2AueMt5zJ9OMA-pgETHZExlA5s0cCzI'}) # age
# last_weight_file.GetContentFile('last_weights.mat')
# model.load_weights('last_weights.mat')

# Compile model
model.compile(optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])

# define 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=37)
cvscores = []

# loop over folds and aggregate metrics
fold_ctr = 1
for train, vald in kfold.split(X, y):
  y1 = np_utils.to_categorical(y, num_cats)
  # Fit the model
  model.fit(X[train], y1[train], epochs=2, batch_size=30, verbose=0)

  # evaluate the model
  scores = model.evaluate(X[vald], y1[vald], verbose=0)
  print("Fold :%d    %s: %.2f%%" % (fold_ctr, model.metrics_names[1], scores[1]*100))
  fold_ctr = fold_ctr + 1
  cvscores.append(scores[1] * 100)
  
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

print('=========<< [4] Predict >>=========')

predictions = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

# Save Keras ModelCheckpoints on google drive

# save model weights
id_64 = '1HQ2-ZE4wmY6KHy5DRHLAsEw-R4UyjgmV'
id_128 = '1nkk_tv_e4aVM_DXS5152smOm3vN55ZZZ'

fname = 'model_weights_age_strat_Kfold.h5'
model.save_weights(fname)
weights_file_aug = drive.CreateFile({'title' : fname, 
                   'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

weights_file_aug.SetContentFile(fname)
weights_file_aug.Upload()
drive.CreateFile({'id': weights_file_aug.get('id')})

# save whole model (architecture + weights + optimizer state)
fname1 = 'model_age_stratified_Kfold.h5'
model.save(fname1)
model_file = drive.CreateFile({'title' : fname1, 
                  'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

model_file.SetContentFile(fname1)
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

y_pred = model.predict_classes(x=X_test, batch_size=None, verbose=0, steps=None)
y_test = np.argmax(y_test, 1)

class_names = ['0-3', '4-7', '8-14', '15-24', '25-37', '38-47', '48-59', '60-']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Age Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Age Normalized confusion matrix')

plt.show()
print("\nelapsed_time = ", (time.time() - start_time)/60, " minutes")


# References:
#     [1] Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
#     [2] https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/



In [0]:
# [2]
# Gender Classifier with stratified Kfold cross validation, UTK
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt
import time

from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix
  
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def stratified_sampling(X, y, test_size, random_state):
    # apply Scikit stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in sss.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test

print("****************<< Gender Classifier >>****************\n")
start_time = time.time()

print("=========<< [1] Retrieving Data >>=========")

# Initialize GoogleDriveFile instance with file id.
data_file = drive.CreateFile({'id': '1BE5HdE_mYkd6XjJ7e_pY6m3nfGaRZ87_'})      # input image (64, 64, 3)
# data_file = drive.CreateFile({'id': '1Tsf7XwB09dUZVXSmlO81Fw3lumUnRiNb'})    # input image (128, 128, 3)
data_file.GetContentFile('UTKdata.mat')

num_cats = 2
sz = 64

data = scipy.io.loadmat('UTKdata')
X = data['X']
y = data['y_gender'][0]

print('=========<< [2] Splitting data into train, validate & test >>=========')

input_shape = (sz, sz, 3)
X, X_test, y, y_test = stratified_sampling(X, y, test_size=0.15, random_state=37)
y_test = np_utils.to_categorical(y_test, num_cats)

print("Train size = ", y.shape[0]) 
print("Test size = ", y_test.shape[0])

print('=========<< [3] Model build & train >>=========')

# create model
vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)
model = Sequential()
for new_layer in vgg16_model.layers:
    model.add(new_layer)
# print(model.summary())

# # load weigts trained on a previus training session
# last_weight_file = drive.CreateFile({'id': '1QRUpGK-kEbw1zylZhpEjXx-ApxBw3LGA'}) # age
# last_weight_file.GetContentFile('last_weights.mat')
# model.load_weights('last_weights.mat')

# Compile model
model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])

# define 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=37)
cvscores = []

# loop over folds and aggregate metrics
fold_ctr = 1
for train, vald in kfold.split(X, y):
  y1 = np_utils.to_categorical(y, num_cats)
  # Fit the model
  model.fit(X[train], y1[train], epochs=2, batch_size=30, verbose=0)

  # evaluate the model
  scores = model.evaluate(X[vald], y1[vald], verbose=0)
  print("Fold :%d  %s = %.2f%%"  % (fold_ctr, model.metrics_names[1], scores[1]*100))
  fold_ctr = fold_ctr + 1
  cvscores.append(scores[1] * 100)
  
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

print('=========<< [4] Predict >>=========')

predictions = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

# Save Keras ModelCheckpoints on google drive

# save model weights
id_64 = '1HQ2-ZE4wmY6KHy5DRHLAsEw-R4UyjgmV'
id_128 = '1nkk_tv_e4aVM_DXS5152smOm3vN55ZZZ'

fname = 'model_weights_gender_strat_Kfold.h5'
model.save_weights(fname)
weights_file_aug = drive.CreateFile({'title' : fname, 
                   'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

weights_file_aug.SetContentFile(fname)
weights_file_aug.Upload()
drive.CreateFile({'id': weights_file_aug.get('id')})

# save whole model (architecture + weights + optimizer state)
fname1 = 'model_gender_stratified_Kfold.h5'
model.save(fname1)
model_file = drive.CreateFile({'title' : fname1, 
                  'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

model_file.SetContentFile(fname1)
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

y_pred = model.predict_classes(x=X_test, batch_size=None, verbose=0, steps=None)
y_test = np.argmax(y_test, 1)

class_names = ['Male', 'Female']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Gender Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Gender Normalized confusion matrix')

plt.show()
print("\nelapsed_time = ", (time.time() - start_time)/60, " minutes")

# References:
#     [1] Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
#     [2] https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

In [0]:
#[3]
# Age Classifier with stratified Kfold cross validation for Adience Benchmark
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt
import time

from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def stratified_sampling(X, y, test_size, random_state):
    # apply Scikit stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in sss.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test
  
print("###############<< Age Classifier >>###############\n")
start_time = time.time()

print("=========<< [1] Retrieving Data >>=========")
print("Adience : 64 ===> Age classification")
# Initialize GoogleDriveFile instance with file id.

# data_file_64 = drive.CreateFile({'id': '1BE5HdE_mYkd6XjJ7e_pY6m3nfGaRZ87_'})
# data_file_64.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_128 = drive.CreateFile({'id': '1Tsf7XwB09dUZVXSmlO81Fw3lumUnRiNb'})
# data_file_128.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_200 = drive.CreateFile({'id': '1hTnC0V1-P0sT-nLhl5JK7ETMpXdL9zgd'})
# data_file_200.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# load Adience data to be used for testing
train_data_64_adience = drive.CreateFile({'id': '1tYfWhBVc9iWKqd2Qgp1It5JmLg6PqPTj'})
train_data_64_adience.GetContentFile('Adience_train_data.mat') 
test_data_64_adience = drive.CreateFile({'id': '1yvBD7UvYMybgeR4smEtLkxxDN8q8kdFo'})
test_data_64_adience.GetContentFile('Adience_test_data.mat')

# train_data_128_adience = drive.CreateFile({'id': '17uW_i5i4QIsbka_QPE424B2OyZgKE7jV'})
# train_data_128_adience.GetContentFile('Adience_train_data.mat') 
# test_data_128_adience = drive.CreateFile({'id': '1nFm9J_rmZpI6ToVBeLrp1GPgXWVnamZT'})
# test_data_128_adience.GetContentFile('Adience_test_data.mat') 

num_cats = 8 # number of age categories (0, 4), [4, 8), [8, 15), [15, 25), [25, 38), [38, 48), [48, 60), 60+
sz = 64     # image dimension

print('=========<< [2] Splitting data into train, validate & test >>=========')

train_data = scipy.io.loadmat('Adience_train_data')
test_data = scipy.io.loadmat('Adience_test_data')
X = train_data['X']
y = train_data['y_age'][0]
X_test = test_data['X']
y_test = test_data['y_age'][0]

input_shape = (sz, sz, 3)
y_test = np_utils.to_categorical(y_test, num_cats)

print("Train size = ", y.shape[0]) 
print("Test size = ", y_test.shape[0])

print('=========<< [3] Model build & train >>=========')

# create model
vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)
model = Sequential()
for new_layer in vgg16_model.layers:
    model.add(new_layer)
# print(model.summary())

# # load weigts trained on a previus training session
# last_weight_file = drive.CreateFile({'id': '1-2AueMt5zJ9OMA-pgETHZExlA5s0cCzI'}) # age
# last_weight_file.GetContentFile('last_weights.mat')
# model.load_weights('last_weights.mat')

# Compile model
model.compile(optimizer=SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])

# define 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=False, random_state=37)
cvscores = []

# loop over folds and aggregate metrics
fold_ctr = 1
for train, vald in kfold.split(X, y):
  y1 = np_utils.to_categorical(y, num_cats)
  # Fit the model
  model.fit(X[train], y1[train], epochs=2, batch_size=30, verbose=0)

  # evaluate the model
  scores = model.evaluate(X[vald], y1[vald], verbose=0)
  print("Fold:%d  %s: %.2f%%" % (fold_ctr, model.metrics_names[1], scores[1]*100))
  fold_ctr = fold_ctr + 1
  cvscores.append(scores[1] * 100)
  
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

print('=========<< [4] Predict >>=========')

predictions = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

y_pred = model.predict_classes(x=X_test, batch_size=None, verbose=0, steps=None)
y_test = np.argmax(y_test, 1)

print("\nelapsed_time = ", (time.time() - start_time)/60, " minutes")

# Compute confusion matrix
# class_names = ['0-3', '4-7', '8-14', '15-24', '25-37', '38-47', '48-59', '60-']
class_names = ['Male', 'Female']
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Age Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Age Normalized confusion matrix')

plt.show()

# Save Keras ModelCheckpoints on google drive

# save model weights
id_64 = '1adEZUwik9dqAcAtyRp3nhjn2um93LGdr'
id_128 = '1ue4YR7spae8QqpCqB9JwHEMEArYS1nQf'

fname = 'model_weights_age_strat_Kfold.h5'
model.save_weights(fname)
weights_file_aug = drive.CreateFile({'title' : fname, 
                   'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

weights_file_aug.SetContentFile(fname)
weights_file_aug.Upload()
drive.CreateFile({'id': weights_file_aug.get('id')})

# save whole model (architecture + weights + optimizer state)
fname1 = 'model_age_stratified_Kfold.h5'
model.save(fname1)
model_file = drive.CreateFile({'title' : fname1, 
                  'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

model_file.SetContentFile(fname1)
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

# References:
#     [1] Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
#     [2] https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

In [0]:
# [4]
# Age Classifier, Data augmentation, Data generator, stratified sampling
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt
import time

from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def stratified_sampling(X, y, test_size, random_state):

    # apply Scikit stratified sampling
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_index, test_index in sss.split(X, y):
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test
  
print("###############<< Age Classifier >>###############\n")
start_time = time.time()

print("=========<< [1] Retrieving Data >>=========")

# Initialize GoogleDriveFile instance with file id.

data_file_64 = drive.CreateFile({'id': '1BE5HdE_mYkd6XjJ7e_pY6m3nfGaRZ87_'})
data_file_64.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_128 = drive.CreateFile({'id': '1Tsf7XwB09dUZVXSmlO81Fw3lumUnRiNb'})
# data_file_128.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# data_file_200 = drive.CreateFile({'id': '1hTnC0V1-P0sT-nLhl5JK7ETMpXdL9zgd'})
# data_file_200.GetContentFile('UTKdata.mat') # Download file as 'UTKdata.mat'.

# train_data_128_adience = drive.CreateFile({'id': '17uW_i5i4QIsbka_QPE424B2OyZgKE7jV'})
# train_data_128_adience.GetContentFile('Adience_train_data.mat') 

# test_data_128_adience = drive.CreateFile({'id': '1nFm9J_rmZpI6ToVBeLrp1GPgXWVnamZT'})
# test_data_128_adience.GetContentFile('Adience_test_data.mat') 

num_cats = 8 # number of age categories (0, 4), [4, 8), [8, 15), [15, 25), [25, 38), [38, 48), [48, 60), 60+
sz = 64     # image dimension
epochs=40

data = scipy.io.loadmat('UTKdata')
X = data['X']
y_age = data['y_age'][0]
y_age = np_utils.to_categorical(y_age, num_cats)

print('=========<< [2] Splitting data into train, validate & test >>=========')

y = y_age

input_shape = (sz, sz, 3)

test_ratio, vald_ratio = 0.15, 0.10
train_batch_size = 30

X_train, X_test, y_train, y_test = stratified_sampling(X, y, test_size=test_ratio, random_state=37)
X_train, X_vald, y_train, y_vald = stratified_sampling(X_train, y_train, test_size=vald_ratio, random_state=37)
print("Train size = ", y_train.shape[0]) 
print("Validation size = ", y_vald.shape[0])
print("Test size = ", y_test.shape[0])

print('=========<< [3] Data augmentation and batch generation >>=========')

vald_batch_size = int(train_batch_size*vald_ratio)
test_batch_size = int(train_batch_size*test_ratio)

# data augmentation
datagen = ImageDataGenerator( featurewise_center=True,
                              featurewise_std_normalization=True,
                              rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)
datagen.fit(X_train)
# datagen = ImageDataGenerator()

train_batches = datagen.flow(x=X_train, y=y_train, batch_size=train_batch_size)
vald_batches = datagen.flow(x=X_vald, y=y_vald, batch_size=vald_batch_size)
test_batches = datagen.flow(x=X_test, y=y_test, batch_size=test_batch_size)

# build, compile and train the model
print('=========<< [4] Model build & train >>=========')

vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)

model = Sequential()

for new_layer in vgg16_model.layers:
    model.add(new_layer)

# print(model.summary())

model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])

# last_weight_file = drive.CreateFile({'id': '1QRUpGK-kEbw1zylZhpEjXx-ApxBw3LGA'}) # age
# last_weight_file.GetContentFile('last_weights.mat')
# model.load_weights('last_weights.mat')

steps_per_epoch = math.ceil( len(X_train) / (train_batch_size) )
vald_steps = math.ceil( len(X_vald) / vald_batch_size )

train_vald_res = model.fit_generator(   generator=train_batches,
                                        steps_per_epoch=steps_per_epoch,
                                        validation_data=vald_batches,
                                        validation_steps=vald_steps,
                                        epochs=epochs,
                                        verbose=2)
print(train_vald_res)
print('=========<< [5] Predict >>=========')

predictions = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

# Save Keras ModelCheckpoints on google drive

# save model weights
id_64 = '1HQ2-ZE4wmY6KHy5DRHLAsEw-R4UyjgmV'
id_128 = '1nkk_tv_e4aVM_DXS5152smOm3vN55ZZZ'

fname = 'model_weights_age.h5'
model.save_weights(fname)
weights_file_aug = drive.CreateFile({'title' : fname, 
                   'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

weights_file_aug.SetContentFile(fname)
weights_file_aug.Upload()
drive.CreateFile({'id': weights_file_aug.get('id')})

# save whole model (architecture + weights + optimizer state)
fname1 = 'model_age.h5'
model.save(fname1)
model_file = drive.CreateFile({'title' : fname1, 
                  'parents': [{'kind': 'drive#fileLink', 'id': id_64}]})

model_file.SetContentFile(fname1)
model_file.Upload()
drive.CreateFile({'id': model_file.get('id')})

y_pred = model.predict_classes(x=X_test, batch_size=None, verbose=0, steps=None)
y_test = np.argmax(y_test, 1)

class_names = ['0-3', '4-7', '8-14', '15-24', '25-37', '38-47', '48-59', '60-']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Age Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Age Normalized confusion matrix')

plt.show()

print("\nelapsed_time = ", (time.time() - start_time)/60, " minutes")

# Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

In [0]:
# [5]
# Age Classifier, random split
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import h5py
  
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("###############<< Age Classifier >>###############\n")

print("=========<< [1] Retrieving Data >>=========")

# Initialize GoogleDriveFile instance with file id.

# data_file_64 = drive.CreateFile({'id': '1l-v3-P-3yncMIg8EJ5EuZWgGDR_QR_Mj'})
# data_file_64.GetContentFile('UTKdata.mat')                                    # Download file as 'UTKdata.mat'.

data_file_128 = drive.CreateFile({'id': '1cU4oHHsRnH2mE2R3tykEQ1gnAWBX3475'})
data_file_128.GetContentFile('UTKdata.mat')                                   # Download file as 'UTKdata.mat'.

# data_file_224 = drive.CreateFile({'id': '1WZM8pWCNO1MLkmomGuwaHjxOrb5MGmO0'})
# data_file_224.GetContentFile('UTKdata.mat')                                 # Download file as 'UTKdata.mat'.

# print('title: %s\n' % (data_file_128['title']))

num_cats = 8 # number of age categories (0, 4), [4, 8), [8, 15), [15, 25), [25, 38), [38, 48), [48, 60), 60+
sz = 128     # image dimension
epochs=10

data = scipy.io.loadmat('UTKdata')
X = data['X']
y_age = data['y_age'][0]
y_age = np_utils.to_categorical(y_age, num_cats)

print('=========<< [2] Splitting data into train, validate & test >>=========')

y = y_age

input_shape = (sz, sz, 3)

test_vald_ratio = 0.20
test_ratio, vald_ratio = 0.10, 0.10
train_batch_size = 30

X_train, X_test_vald, y_train, y_test_vald = train_test_split(X, y, test_size=test_vald_ratio, random_state=37)
X_vald, X_test, y_vald, y_test = train_test_split(X_test_vald, y_test_vald, test_size = 0.50, random_state=37)

print('=========<< [3] Data augmentation and batch generation >>=========')

vald_batch_size = int(train_batch_size*vald_ratio)
test_batch_size = int(train_batch_size*test_ratio)

# data augmentation
# datagen = ImageDataGenerator( featurewise_center=True,
#                               featurewise_std_normalization=True,
#                               rotation_range=20,
#                               width_shift_range=0.2,
#                               height_shift_range=0.2,
#                               horizontal_flip=True)
# datagen.fit(X_train)

datagen = ImageDataGenerator()
train_batches = datagen.flow(x=X_train, y=y_train, batch_size=train_batch_size)
vald_batches =datagen.flow(x=X_vald, y=y_vald, batch_size=vald_batch_size)
test_batches = datagen.flow(x=X_test, y=y_test, batch_size=test_batch_size)

# build, compile and train the model
print('=========<< [4] Model build & train >>=========')

vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)

model = Sequential()

for new_layer in vgg16_model.layers:
    model.add(new_layer)

print(model.summary())
model.compile(optimizer=SGD(lr=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])
last_weight_file = drive.CreateFile({'id': '1Tsu1ogHumh1HHE8pTjl37nhtoqAWf_7P'}) # age
last_weight_file.GetContentFile('last_weights.mat')
model.load_weights('last_weights.mat')

steps_per_epoch = math.ceil( len(X_train) / (train_batch_size) )
vald_steps = math.ceil( len(X_vald) / vald_batch_size )

train_vald_res = model.fit_generator(   generator=train_batches,
                                        steps_per_epoch=steps_per_epoch,
                                        validation_data=vald_batches,
                                        validation_steps=vald_steps,
                                        epochs=epochs,
                                        verbose=2)
print(train_vald_res)
print('=========<< [5] Predict >>=========')

predictions = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

# Save Keras ModelCheckpoints on google drive

# fname = 'model_64_weights_aug_age.h5'
# model.save_weights(fname)
# weights_file_aug = drive.CreateFile({'title' : fname,
#                     'parents': [{'kind': 'drive#fileLink', 'id': '1tQbF4hNKhher8BnnUWq9YaoIovjoZM6E'}] })

fname = 'model_128_weights_age.h5'
model.save_weights(fname)
weights_file_aug = drive.CreateFile({'title' : fname,
                    'parents': [{'kind': 'drive#fileLink', 'id': '109-8ppwvlItSn8U-UuCgSRjNEu799ETG'}] })

weights_file_aug.SetContentFile(fname)
weights_file_aug.Upload()
# print('Uploaded file with ID {}'.format(weights_file_aug.get('id')))
drive.CreateFile({  'id': weights_file_aug.get('id')})

In [0]:
# [6]
# testing the trained model on different data set
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from keras.models import load_model
from keras.utils import np_utils

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

num_cats = 8

# load Adience data to be used for testing
# train_data_64_adience = drive.CreateFile({'id': '1tYfWhBVc9iWKqd2Qgp1It5JmLg6PqPTj'})
# train_data_64_adience.GetContentFile('Adience_train_data.mat') 
test_data_64_adience = drive.CreateFile({'id': '1yvBD7UvYMybgeR4smEtLkxxDN8q8kdFo'})
test_data_64_adience.GetContentFile('Adience_test_data.mat')

# train_data_128_adience = drive.CreateFile({'id': '17uW_i5i4QIsbka_QPE424B2OyZgKE7jV'})
# train_data_128_adience.GetContentFile('Adience_train_data.mat') 
# test_data_128_adience = drive.CreateFile({'id': '1nFm9J_rmZpI6ToVBeLrp1GPgXWVnamZT'})
# test_data_128_adience.GetContentFile('Adience_test_data.mat') 

# dataAd_train = scipy.io.loadmat('Adience_train_data')
dataAd_test = scipy.io.loadmat('Adience_test_data')

# X1 = dataAd_train['X']
# y1_age = dataAd_train['y_age'][0]
# y1_age_cat = np_utils.to_categorical(y1_age, num_cats)

X2 = dataAd_test['X']
y2_age = dataAd_test['y_age'][0]
y2_age_cat = np_utils.to_categorical(y2_age, num_cats)

# returns a compiled model
last_model_file = drive.CreateFile({'id': '1Y9LwSYXUBEEx4DunWPRzDX3Jnq7MKw0W'}) # age (64, 64, 3)
last_model_file.GetContentFile('last_model.h5')
model = load_model('last_model.h5')

predictions = model.evaluate(x=X2, y=y2_age_cat, verbose=2)
print("test_loss = ", predictions[0], " test_accuracy = ", predictions[1])

y_pred = model.predict_classes(x=X2, batch_size=None, verbose=0, steps=None)

class_names = ['0-3', '4-7', '8-14', '15-24', '25-37', '38-47', '48-59', '60-']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y2_age, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Age Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Age Normalized confusion matrix')

plt.show()

# Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.


In [0]:
# [7]
# Age Classifier, combined datasets (Not implemented)
!pip install -U -q PyDrive
  
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

import keras
import math
import scipy.io
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

print("###############<< Age Classifier >>###############\n")

print("=========<< [1] Retrieving Data >>=========")

# Initialize GoogleDriveFile instance with file id.

data_utk = drive.CreateFile({'id': '1BE5HdE_mYkd6XjJ7e_pY6m3nfGaRZ87_'})
data_utk.GetContentFile('UTK_data.mat')

train_data_adience = drive.CreateFile({'id': '1tYfWhBVc9iWKqd2Qgp1It5JmLg6PqPTj'})
train_data_adience.GetContentFile('Adience_train_data.mat') 

test_data_adience = drive.CreateFile({'id': '1yvBD7UvYMybgeR4smEtLkxxDN8q8kdFo'})
test_data_adience.GetContentFile('Adience_test_data.mat')

num_cats = 8 # number of age categories (0, 4), [4, 8), [8, 15), [15, 25), [25, 38), [38, 48), [48, 60), 60+
sz = 64     # image dimension
epochs=1

dataUTK = scipy.io.loadmat('UTK_data')
dataAd_train = scipy.io.loadmat('Adience_train_data')
dataAd_test = scipy.io.loadmat('Adience_test_data')

X = dataUTK['X']
y_age = dataUTK['y_age'][0]
y_age = np_utils.to_categorical(y_age, num_cats)

X1 = dataAd_train['X']
y1_age = dataAd_train['y_age'][0]
y1_age = np_utils.to_categorical(y1_age, num_cats)

X2 = dataAd_test['X']
y2_age = dataAd_test['y_age'][0]
y2_age = np_utils.to_categorical(y2_age, num_cats)

print('=========<< [2] Splitting data into train, validate & test >>=========')

y = y_age

input_shape = (sz, sz, 3)

test_ratio, vald_ratio = 0.15, 0.10
train_batch_size = 50

# extract testing data from UTK data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=37)

# concatenate UTK and Adience test data
X_test = np.concatenate([X_test, X2])
y_test = np.concatenate([y_test, y2_age])

# concatenate UTK and Adience train data
X_train = np.concatenate([X_train, X1])           
y_train = np.concatenate([y_train, y1_age])

# extract validation data from combined train data
X_train, X_vald, y_train, y_vald = train_test_split(X_train, y_train, test_size = vald_ratio, random_state=37)

print('=========<< [3] Data augmentation and batch generation >>=========')

vald_batch_size = int(train_batch_size*vald_ratio)
test_batch_size = int(train_batch_size*test_ratio)

# data augmentation
datagen = ImageDataGenerator( featurewise_center=True,
                              featurewise_std_normalization=True,
                              rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)
datagen.fit(X_train)

# datagen = ImageDataGenerator()
train_batches = datagen.flow(x=X_train, y=y_train, batch_size=train_batch_size)
vald_batches = datagen.flow(x=X_vald, y=y_vald, batch_size=vald_batch_size)
test_batches = datagen.flow(x=X_test, y=y_test, batch_size=test_batch_size)

# build, compile and train the model
print('=========<< [4] Model build & train >>=========')

vgg16_model = keras.applications.vgg16.VGG16(input_shape=input_shape, weights=None, classes=num_cats)

model = Sequential()

for new_layer in vgg16_model.layers:
    model.add(new_layer)

print(model.summary())
model.compile(optimizer=SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True),
              loss='categorical_crossentropy', metrics=['accuracy'])
# last_weight_file = drive.CreateFile({'id': '13UdI_LwEfrPDzn4NAIZnGnlne_Z5rJOo'}) # age
# last_weight_file.GetContentFile('last_weights.mat')
# model.load_weights('last_weights.mat')

steps_per_epoch = math.ceil( len(X_train) / (train_batch_size) )
vald_steps = math.ceil( len(X_vald) / vald_batch_size )

train_vald_res = model.fit_generator(   generator=train_batches,
                                        steps_per_epoch=steps_per_epoch,
                                        validation_data=vald_batches,
                                        validation_steps=vald_steps,
                                        epochs=epochs,
                                        verbose=2)
print(train_vald_res)
print('=========<< [5] Predict >>=========')

eval = model.evaluate(x=X_test, y=y_test, verbose=2)
print("test_loss = ", eval[0], " test_accuracy = ", eval[1])

# # Save Keras ModelCheckpoints on google drive
# fname = 'model_128_weights_age.h5'
# model.save_weights(fname)
# weights_file_aug = drive.CreateFile({'title' : fname, 
#                    'parents': [{'kind': 'drive#fileLink', 'id': '1FUSWSUXc3AYImqTsrZNmQJ0A9vJQb9GO'}]})

# weights_file_aug.SetContentFile(fname)
# weights_file_aug.Upload()
# drive.CreateFile({'id': weights_file_aug.get('id')})

y_pred = model.predict(x=X_test, batch_size=None, verbose=0, steps=None)

y_test = np.argmax(y_test, 1)

class_names = ['0-', '4-', '8-', '15-', '25-', '38-', '48-', '60-']

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Age Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Age Normalized confusion matrix')

plt.show()

# Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.